# Machine Learning Model for Subway Dataset

### Imports:

In [2]:
import pandas as pd

### Loading in Weather Data:


### Loading in Subway Data:


In [3]:
turnstile_data=pd.read_csv('cleaned_turnstile_data.csv')
    

In [4]:
turnstile_data.head()

,id,turnstile_id,remote_unit_id,time_stamp,station_name,entries,exits
0,R138 R293 00-03-03 1546290000,R138 R293 00-03-03,R293,1546290000,34 ST-PENN STA,0,0
1,R138 R293 00-02-05 1546290000,R138 R293 00-02-05,R293,1546290000,34 ST-PENN STA,0,0
2,R138 R293 00-02-03 1546290000,R138 R293 00-02-03,R293,1546290000,34 ST-PENN STA,0,0
3,R138 R293 00-00-02 1546290000,R138 R293 00-00-02,R293,1546290000,34 ST-PENN STA,0,0
4,R138 R293 00-03-01 1546290000,R138 R293 00-03-01,R293,1546290000,34 ST-PENN STA,0,0


### Creating busyness percentage for each station
- For each remote unit id and time condense to one row relating to busyness of that station at that time
- The busyness of a place is how many people are currently there, so exits-entries along with the magnitude in the sum of both values, can do both
- Sum of entries and exits for each remote unit id at that time
- Exits - entries for each remote unit id at that time
- Normalize each of these values

In [5]:
grouped_turnstile_data=turnstile_data.groupby(['remote_unit_id', 'time_stamp']).agg({'entries': 'max', 'exits': 'max'}).reset_index()

grouped_turnstile_data.head()

,remote_unit_id,time_stamp,entries,exits
0,R001,1546300800,0,0
1,R001,1546315200,102,104
2,R001,1546329600,96,51
3,R001,1546344000,260,476
4,R001,1546358400,540,496


In [6]:
len(grouped_turnstile_data)

297994

In [7]:
#add two new columns, one for difference between exits and entries as a proxy for surplus people in that area
#the other for the addition of exits and entries as a proxy for busyness in that area by volume
grouped_turnstile_data['num_people_exited']=grouped_turnstile_data['exits']-grouped_turnstile_data['entries']
#remove negative values and set to zero
#grouped_turnstile_data['num_people_exited']=grouped_turnstile_data['num_people_exited'].apply(lambda x: 0 if x<0 else x)

grouped_turnstile_data['sum_exit_entry']=grouped_turnstile_data['exits']+grouped_turnstile_data['entries']

grouped_turnstile_data.head(10)


,remote_unit_id,time_stamp,entries,exits,num_people_exited,sum_exit_entry
0,R001,1546300800,0,0,0,0
1,R001,1546315200,102,104,2,206
2,R001,1546329600,96,51,-45,147
3,R001,1546344000,260,476,216,736
4,R001,1546358400,540,496,-44,1036
5,R001,1546372800,602,250,-352,852
6,R001,1546387200,159,147,-12,306
7,R001,1546401600,48,38,-10,86
8,R001,1546416000,516,434,-82,950
9,R001,1546430400,842,1351,509,2193


### Use of z-score
- A z-score quantifies the distance between a data point and the mean of a distribution in terms of standard deviations. It indicates how many standard deviations an observation is above or below the mean of a distribution. 

In [8]:
#get a standardized z score for each staiton
from sklearn.preprocessing import StandardScaler

#function to be applied to each group
def standardize_group(group):
    scaler = StandardScaler()
    group[['z_score_num_people_exited', 'z_score_sum_exit_entry']] = scaler.fit_transform(group[['num_people_exited', 'sum_exit_entry']])
    return group

#apply the function to each group
grouped_turnstile_data = grouped_turnstile_data.groupby('remote_unit_id', group_keys=True).apply(standardize_group)

grouped_turnstile_data.head(20)


remote_unit_id  time_stamp  entries  exits  \
remote_unit_id                                                 
R001           0            R001  1546300800        0      0   
               1            R001  1546315200      102    104   
               2            R001  1546329600       96     51   
               3            R001  1546344000      260    476   
               4            R001  1546358400      540    496   
               5            R001  1546372800      602    250   
               6            R001  1546387200      159    147   
               7            R001  1546401600       48     38   
               8            R001  1546416000      516    434   
               9            R001  1546430400      842   1351   
               10           R001  1546444800      784    456   
               11           R001  1546459200     1445    686   
               12           R001  1546473600      284    225   
               13           R001  1546488000       68     68   
               14           R001  1546502400      486    418   
               15           R001  1546516800      887   1370   
               16           R001  1546531200      776    482   
               17           R001  1546545600     1526    754   
               18           R001  1546560000      292    243   
               19           R001  1546574400       85     72   

                   num_people_exited  sum_exit_entry  \
remote_unit_id                                         
R001           0                   0               0   
               1                   2             206   
               2                 -45             147   
               3                 216             736   
               4                 -44            1036   
               5                -352             852   
               6                 -12             306   
               7                 -10              86   
               8                 -82             950   
               9                 509            2193   
               10               -328            1240   
               11               -759            2131   
               12                -59             509   
               13                  0             136   
               14                -68             904   
               15                483            2257   
               16               -294            1258   
               17               -772            2280   
               18                -49             535   
               19                -13             157   

                   z_score_num_people_exited  z_score_sum_exit_entry  
remote_unit_id                                                        
R001           0                    0.031249               -0.917244  
               1                    0.039109               -0.687580  
               2                   -0.145608               -0.753358  
               3                    0.880160               -0.096695  
               4                   -0.141677                0.237767  
               5                   -1.352161                0.032630  
               6                   -0.015913               -0.576092  
               7                   -0.008053               -0.821365  
               8                   -0.291023                0.141888  
               9                    2.031692                1.527679  
               10                  -1.257838                0.465202  
               11                  -2.951729                1.458557  
               12                  -0.200630               -0.349772  
               13                   0.031249               -0.765621  
               14                  -0.236001                0.090604  
               15                   1.929508                1.599031  
               16                  -1.124213                0.485270  
               17          

In [9]:
#convert the z-score to a percentile, we want % busyness after all
from scipy.stats import norm

grouped_turnstile_data['percentile_num_people_exited'] = norm.cdf(grouped_turnstile_data['z_score_num_people_exited'])

grouped_turnstile_data['percentile_sum_exit_entry'] = norm.cdf(grouped_turnstile_data['z_score_sum_exit_entry'])

grouped_turnstile_data.head(10)

remote_unit_id  time_stamp  entries  exits  \
remote_unit_id                                                
R001           0           R001  1546300800        0      0   
               1           R001  1546315200      102    104   
               2           R001  1546329600       96     51   
               3           R001  1546344000      260    476   
               4           R001  1546358400      540    496   
               5           R001  1546372800      602    250   
               6           R001  1546387200      159    147   
               7           R001  1546401600       48     38   
               8           R001  1546416000      516    434   
               9           R001  1546430400      842   1351   

                  num_people_exited  sum_exit_entry  \
remote_unit_id                                        
R001           0                  0               0   
               1                  2             206   
               2                -45             147   
               3                216             736   
               4                -44            1036   
               5               -352             852   
               6                -12             306   
               7                -10              86   
               8                -82             950   
               9                509            2193   

                  z_score_num_people_exited  z_score_sum_exit_entry  \
remote_unit_id                                                        
R001           0                   0.031249               -0.917244   
               1                   0.039109               -0.687580   
               2                  -0.145608               -0.753358   
               3                   0.880160               -0.096695   
               4                  -0.141677                0.237767   
               5                  -1.352161                0.032630   
               6                  -0.015913               -0.576092   
               7                  -0.008053               -0.821365   
               8                  -0.291023                0.141888   
               9                   2.031692                1.527679   

                  percentile_num_people_exited  percentile_sum_exit_entry  
remote_unit_id                                                             
R001           0                      0.512464                   0.179507  
               1                      0.515598                   0.245859  
               2                      0.442116                   0.225618  
               3                      0.810614                   0.461484  
               4                      0.443667                   0.593969  
               5                      0.088162                   0.513015  
               6                      0.493652                   0.282276  
               7                      0.496788                   0.205719  
               8                      0.385517                   0.556416  
               9                      0.978908                   0.936704

In [10]:
df_busyness=grouped_turnstile_data[['remote_unit_id', 'time_stamp', 'entries', 'exits','percentile_num_people_exited','percentile_sum_exit_entry']].copy()
df_busyness.tail(10)

remote_unit_id  time_stamp  entries  exits  \
remote_unit_id                                                     
R572           297984           R572  1577347200      214    200   
               297985           R572  1577361600      523    393   
               297986           R572  1577376000      476    400   
               297987           R572  1577390400      540    635   
               297988           R572  1577404800      119    379   
               297989           R572  1577419200       29     80   
               297990           R572  1577433600      237    212   
               297991           R572  1577448000      631    438   
               297992           R572  1577462400      551    500   
               297993           R572  1577476800      586    612   

                       percentile_num_people_exited  percentile_sum_exit_entry  
remote_unit_id                                                                  
R572           297984                      0.543978                   0.273188  
               297985                      0.346874                   0.514420  
               297986                      0.436817                   0.494099  
               297987                      0.720480                   0.642825  
               297988                      0.903456                   0.309871  
               297989                      0.652843                   0.160684  
               297990                      0.524975                   0.288209  
               297991                      0.252186                   0.591349  
               297992                      0.479913                   0.582421  
               297993                      0.611923                   0.653694

### Explain the new columns:
- Percentile_num_people_exited is a normalized percentage for each station (remote unit id). A value of ~0.5 indicates the same number of entries and exits. A value lower than ~0.5 indicates more entries than exits. A value higher than ~0.5 indicates more exits than entries.
- I see this as a useful metric as the more exits there are than entries the busier the surrounding area is likely to be.
- This column can be used for a more general busyness prediction of areas around stations.
- Percentile_sum_exit_entry is a normalized percentage for each station (remote unit id). The higher the value the greater the total entries and exits in that time period.
- I see this as a useful metric as it is a proxy for the footfall in a station at a certain time.
- This column can be used for a more specific busyness prediciton of each station.



#### Tidy up the columns and names

In [11]:
#change the new columns to percentages and 4 digits
df_busyness['percentile_num_people_exited']=df_busyness['percentile_num_people_exited'].apply(lambda x: round(x*100,2))
df_busyness['percentile_sum_exit_entry']=df_busyness['percentile_sum_exit_entry'].apply(lambda x: round(x*100,2))
#change the names of the new columns
df_busyness.rename(columns={'percentile_num_people_exited':'surrounding_area_busyness', 'percentile_sum_exit_entry':'station_busyness'}, inplace=True)
df_busyness.head(10)

remote_unit_id  time_stamp  entries  exits  \
remote_unit_id                                                
R001           0           R001  1546300800        0      0   
               1           R001  1546315200      102    104   
               2           R001  1546329600       96     51   
               3           R001  1546344000      260    476   
               4           R001  1546358400      540    496   
               5           R001  1546372800      602    250   
               6           R001  1546387200      159    147   
               7           R001  1546401600       48     38   
               8           R001  1546416000      516    434   
               9           R001  1546430400      842   1351   

                  surrounding_area_busyness  station_busyness  
remote_unit_id                                                 
R001           0                      51.25             17.95  
               1                      51.56             24.59  
               2                      44.21             22.56  
               3                      81.06             46.15  
               4                      44.37             59.40  
               5                       8.82             51.30  
               6                      49.37             28.23  
               7                      49.68             20.57  
               8                      38.55             55.64  
               9                      97.89             93.67

In [12]:
df_busyness.tail(20)

remote_unit_id  time_stamp  entries  exits  \
remote_unit_id                                                     
R572           297974           R572  1577203200      570    460   
               297975           R572  1577217600      407    532   
               297976           R572  1577232000      144    260   
               297977           R572  1577246400       25    101   
               297978           R572  1577260800       64     70   
               297979           R572  1577275200      224    157   
               297980           R572  1577289600      315    228   
               297981           R572  1577304000      253    321   
               297982           R572  1577318400      115    243   
               297983           R572  1577332800       23     70   
               297984           R572  1577347200      214    200   
               297985           R572  1577361600      523    393   
               297986           R572  1577376000      476    400   
               297987           R572  1577390400      540    635   
               297988           R572  1577404800      119    379   
               297989           R572  1577419200       29     80   
               297990           R572  1577433600      237    212   
               297991           R572  1577448000      631    438   
               297992           R572  1577462400      551    500   
               297993           R572  1577476800      586    612   

                       surrounding_area_busyness  station_busyness  
remote_unit_id                                                      
R572           297974                      37.95             57.20  
               297975                      76.26             52.61  
               297976                      75.03             26.90  
               297977                      69.21             16.60  
               297978                      57.82             16.86  
               297979                      45.23             25.94  
               297980                      41.81             33.04  
               297981                      67.97             34.48  
               297982                      76.66             25.00  
               297983                      64.64             15.58  
               297984                      54.40             27.32  
               297985                      34.69             51.44  
               297986                      43.68             49.41  
               297987                      72.05             64.28  
               297988                      90.35             30.99  
               297989                      65.28             16.07  
               297990                      52.50             28.82  
               297991                      25.22             59.13  
               297992                      47.99             58.24  
               297993                      61.19             65.37

### For training the model:
- Include the day, time of day and weather for each row

### Map the weather to each station for each time interval

In [13]:
'''df_turn['day'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.day_name()
df_turn['date'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.date
df_turn['time'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.time'''

"df_turn['day'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.day_name()\ndf_turn['date'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.date\ndf_turn['time'] = pd.to_datetime(df_turn['time_stamp'], unit='s').dt.time"